In [1]:
#import sys
#sys.path.append('pyttr')
#from pyttr.ttrtypes import *
from ttrtypes import *
#from pyttr.utils import *
from ttrtypes import *
from utils import show_latex

For comparison, how Arild represented images in his thesis. Feel free to skip over this.

In [2]:
Int = BType('Int')
Ind = BType('Ind')
Image = BType('Image')

Image.learn_witness_condition(lambda x: isinstance(x, PIL.Image.Image))

Segment = RecType({'cx': Int, 'cy': Int, 'w': Int, 'h': Int})
Ppty = FunType(Ind, Ty)
Obj = RecType({'seg': Segment, 'pfun': Ppty})
Objs = ListType(Obj)
ObjDetector = FunType(Image, Objs)

show_latex(ObjDetector)

<IPython.core.display.Latex object>

In [3]:
yolo_out ={'label': 'dog', 'loc': {'w': 80, 'cy': 209, 'h': 70, 'cx': 328}}

In [4]:
dog = Pred('dog',[Ind])

T = Rec({'pfun':(Fun('a',Ind,PType(dog,['a']))),'seg':Rec({'w': 80, 'cy': 209, 'h': 70, 'cx': 328})})
show_latex(T)

#The object in the image is represented with a bounding box and a predicate

<IPython.core.display.Latex object>

But what if I represent the object with the bounding box and an Ind constant?
The idea is to have a list of objects without making any judgements about the predicates that apply to them. 
I'm thinking they should be records (not record types), since they are concrete instances. Does that make sense?

In [5]:
T1 = Rec({'x':'a','seg':Rec({'w': 80, 'cy': 209, 'h': 70, 'cx': 328})})
T2 = Rec({'x':'b','seg':Rec({'w': 40, 'cy': 109, 'h': 20, 'cx': 256})})
Segments = [T1, T2]
show_latex(Segments)

<IPython.core.display.Latex object>

In [6]:
def dog_idfun(T):
    '''placeholder for an actual classifier 
    the actual classifier will make a binary decision based on information from the record'''
    if T.seg.h <50:
        return False
    else:
        return True

In [7]:
[dog_idfun(t) for t in Segments]

[True, False]

In general, I want to use utterances to guide visual classification.

As a simple example, this would be a representation for "There is a dog." The response should be True or False.

In [8]:
Utt = RecType({'x':Ind, 'c':(Fun('v',Ind,PType(dog,['v'])),['x'])})

show_latex(Utt)

<IPython.core.display.Latex object>

Instead of subtype checking, I thought about checking if the image segment is record of the record type of the utterance.

In [9]:
T = Rec({'x':'a', 'c':(Fun('v',Ind,PType(dog,['v'])),['x'])})
show_latex(T)

<IPython.core.display.Latex object>

In principle, I need to do something like the following. I'm stuck on how to integrate the representation of the image segments and the predicate.

The predicate should determine the classifier to use on the image segments.

In [10]:
M = Possibility('M')

#the finished version should do this based on the actual image ofc
Ind.judge('a')
Ind.judge('b')

clsf = dog_idfun
for seg in Segments:
    if clsf(seg):
        dog_a = PType(dog,[seg.x])  #again, the dog needs to be variable in the final version
        dog_a.in_poss(M).judge('s')
        a_is_dog = Rec({'x':seg.x,'c':'s'})

print(Utt.in_poss(M).query(a_is_dog))
print(show(M))
#print(show(dog_a))
show_latex(a_is_dog)


True

M:
_____________________________________________
Ind: [a, b]
dog(a): [s]
_____________________________________________



<IPython.core.display.Latex object>

In [11]:
Utt.in_poss(M).query(a_is_dog)

True

In [12]:
print(RecOfRecType(a_is_dog,Utt,M))

True


### Robin's suggestion

In [40]:
Int.learn_witness_condition(lambda x: isinstance(x,int))
DogSeg = BType('DogSeg')
DogSeg.learn_witness_condition(lambda seg: Segment.query(seg) and seg.h<50)
dog.learn_witness_fun(lambda args: RecType({'x':SingletonType(Ind,args[0]),'seg':DogSeg}))
dog_b = PType(dog,['b'])
print(dog_b.query(T2))
print([PType(dog,[s.x]).query(s) for s in Segments])
Dog_situations = [Rec({'x':s.x,'c':s}) for s in Segments if PType(dog,[s.x]).query(s)]
print(Utt.query(Dog_situations[0]))
show_latex(Dog_situations)

True
[False, True]
True


<IPython.core.display.Latex object>

Note that in the above `Segment` is the type of segments but `Segments` is a list of records containing an individual and a segment.  You might want to change one of the names for clarity.

Your classifier will be called in the definition of the witness condition for the type `DogSeg` (currently represented by `seg.h<50`).